 import libaries, load documents and get roots

In [1]:
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

In [2]:
#get document, and the root of the string documents
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()

Question1: 10 countries with the lowest infant mortality rates

In [3]:
# parse name of the country, infant_mortality rate,
# then update the mortality dictionary

mort = {}
for child in root.iter('country'):
    mortality = child.findtext('infant_mortality')
    name = child.find('name').text
    mort.update({name:mortality})
    

In [4]:
# read dictinary mort as data frame
# sort df by mortality rate, and show lowest 10
mortality_df = pd.DataFrame.from_dict(mort, orient = "index")
mortality_df.columns = ["mortality"]
mortality_df.sort_values("mortality").head(10)

,mortality
Monaco,1.81
Romania,10.16
Fiji,10.2
Brunei,10.48
Grenada,10.5
Mauritius,10.59
Panama,10.7
Seychelles,10.77
United Arab Emirates,10.92
Barbados,10.93


Question2 : 10 cities with the largest population

In [5]:
#parse, city name population
pop_dict = {}
for child in root.iter('city'):
    for pop in child.getiterator('population'):
        population = int(pop.text)
        year = pop.get('year')
        city = child.findtext('name')
        if year == "2011":
            pop_dict.update({city: population})

In [6]:
# read dictionary as data frame
# sort by population, descending order
population_df = pd.DataFrame.from_dict(pop_dict, orient = "index")
population_df.columns = ["population"]
population_df.sort_values("population",ascending= False).head(10)

,population
Mumbai,12442373
Delhi,11034555
Bangalore,8443675
Tehran,8154051
Dhaka,7423137
Hyderabad,6731790
Ahmadabad,5577940
Luanda,5000000
Chennai,4646732
Sydney,4605992


Question 3: 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)


In [7]:
ethn_name = list()
ethn_population = list()

# extract lastest population
# get the population of each ethnicity in every country
# write the ethnicity group name and population into seperate list

for child in root.iter('country'):
    country = child.findtext('name')
    year0 = 1950
    for population in child.findall('population'):
        year = int(population.get('year'))
        if year > year0:
            year0 = year
            pop = int (population.text)
    for ethinicty in root.iter('ethnicgroup'):
        percent = float (ethinicty.get('percentage'))
        ethname = ethinicty.text
        ethpop = percent * pop / 100 
        ethn_name.append(ethname)
        ethn_population.append(ethpop)

In [8]:
#dataframe 
ethn_df = pd.DataFrame({'ethnicity':ethn_name ,'population':ethn_population})

#groupby ethnicity group and sum

ethn_df.groupby('ethnicity').sum().sort_values('population', ascending = False).head(10)

,population
ethnicity,
African,1.302733e+11
European,6.768454e+10
Mestizo,6.070428e+10
Polynesian,4.646768e+10
Arab,4.450160e+10
Chinese,3.297009e+10
Arab-Berber,2.064378e+10
Black,1.781319e+10
Malay,1.689290e+10


Question4: name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [9]:
rivernamels= list()
citynamels = list()
countrynamels = list()
latitudels = list()
longitudels = list()

for child in root.iter('country'):
    country = child.findtext('name')
    for city in root.getiterator('city'):
        cityname = city.findtext('name')
        latitude = city.findtext('latitude')
        longitude = city.findtext('longitude')
        for river in city.findall('located_at'):
            if river.get('watertype') == "river":
                river_name =  river.get('river')
                rivernamels.append(river_name)
                citynamels.append(cityname)
                countrynamels.append(country)
                latitudels.append(latitude)
                longitudels.append(longitude)

In [15]:
river_df = pd.DataFrame({'river':rivernamels ,'city':citynamels, 'country':countrynamels, 'latitude':latitudels,'longitude':longitudels})
river_df

,city,country,latitude,longitude,river
0,Beograd,Albania,44.8,20.4667,river-Donau
1,Beograd,Albania,44.8,20.4667,river-Save
2,Novi Sad,Albania,None,None,river-Donau
3,Niš,Albania,None,None,river-SouthernMorava
4,Podgorica,Albania,42.26,19.16,river-Moraca
5,Strasbourg,Albania,48.5833,7.76667,river-Rhein
6,Bordeaux,Albania,44.51,-0.4,river-Garonne
7,Orléans,Albania,47.5,1.5,river-Loire
8,Tours,Albania,None,None,river-Loire
9,Chalons en Champagne,Albania,49.0,4.4,river-Marne
